# PerfCi CNV Summary Report

This notebook is designed to perform a PerfCI VM workloads comparison between OpenShift versions using the Geometric Mean 
between the 2 most recent major versions.

    * HammerDB: Database TPM [Transaction Per Minute] for Mariadb, MSSQL, PostgreSQL
    * Uperf: Network between two VMs
    * Vdbench: VM Application storage: oltp1, oltp2, oltphw, odss2, odss128
    * Bootstorm: Time till VM login

How to use it:

1. Run all and get the percentage results at the end.
2. If you want different versions instead of the default, choose them and rerun the 'Workloads comparison' cell.
3. The default number of fetch days is 365 days. It can be configured in the prerequisite variable ES_NUM_FETCH_DAYS = 365.

## Prerequsite 

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

import ipywidgets as widgets
from IPython.display import display, HTML
import os
import configparser
config = configparser.ConfigParser()
config.read('data.conf')
es_host = config.get("data", 'es_host') 
es_port = config.get("data", 'es_port')
es_user = config.get("data", 'es_user') 
es_password = config.get("data", 'es_password')
os.environ['grafana_url'] = config.get("data", 'grafana_url')
os.environ['network_speed'] = config.get("data", 'network_speed')
os.environ['fetch_ocp_versions_days'] = config.get("data", 'fetch_ocp_versions_days')
os.environ['filter_kind'] = config.get("data", 'filter_kind')

from benchmark_runner.common.elasticsearch.elasticsearch_operations import ElasticSearchOperations
from benchmark_runner.jupyterlab.templates.summary_report.summary_report_widgets import SummaryReportWidgets

elasticsearch = ElasticSearchOperations(es_host=es_host, es_port=es_port, es_user=es_user, es_password=es_password)
summary_report_widget = SummaryReportWidgets(elasticsearch=elasticsearch)

# Default 365 days for ocp version fetch days 
summary_report_widget.set_fetch_ocp_version_days(num_fetch_days=int(os.environ['fetch_ocp_versions_days']))


## Select OCP versions: [Ctrl/Cmd key]

In [ ]:
# There is an option to choose specific ocp versions instead of default
summary_report_widget.choose_ocp_versions()

SelectMultiple(index=(13, 0), layout=Layout(height='300px'), options=('4.14.10', '4.14.8', '4.14.7', '4.14.6',…

## Workloads comparison

In [ ]:
# There is an option to choose specific workload comparison
report_df = summary_report_widget.analyze_all_workload(workloads = ['hammerdb', 'hammerdb_lso', 'uperf', 'vdbench', 'vdbench_scale', 'bootstorm', 'windows'])
summary_report_widget.display_df(report_df)

,workload,metric,storage type,"('4.13.29', '4.14.10')"
0,hammerdb,TPM (mariadb),ODF,3.58%
1,hammerdb,TPM (mssql),ODF,-0.15%
2,hammerdb,TPM (postgresql),ODF,-0.83%
3,hammerdb,TPM (postgresql),LSO,12.07%
4,uperf,Throughput,Ephemeral,-4.6%
5,uperf,Latency,Ephemeral,2.83%
6,vdbench,Iops,ODF,4.52%
7,vdbench,Latency,ODF,6.15%
8,vdbench_scale,Iops,ODF,27.43%
9,vdbench_scale,Latency,ODF,23.11%


## Comparison details

In [ ]:
summary_report_widget.get_comparison_details()

ocp_version,odf_version,cnv_nightly_version,sample_dates,uuid
4.13.29,[4.12.10-2],[4.13.7-315],"[2024-01-14, 2024-01-16]","[1e48c09b-2833-5724-bad4-3774f5e0a80f, 12476740-9774-56ed-9861-06b3259c4688]"
4.14.10,[4.13.6-1],"[4.14.3-168, 4.14.3-123]","[2024-01-23, 2024-01-20, 2024-01-24, 2024-01-19, 2024-01-26]","[0d9a9919-84ef-584e-b232-5480cc691325, a3c73e8b-f8ab-5d30-ab24-a41a490b663d, 2eba04a1-2293-5ad2-bcc1-5556b427f3de, 25026148-f119-5f28-a6ca-849d72cf9d27, 2c2b5296-cfba-5b6c-9af6-70e15016690e]"


## Upload to ElasticSearch

In [ ]:
summary_report_widget.upload_report_to_elasticsearch(df=report_df, index_name='summary-report')